<h1>LABORATORIO 4 </h1>
<h1>LOAIZA CORONADO RODRIGO JAIRO </h1>
<h2>ING DE SISTEMAS</h2>

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
import pandas as pd

from sklearn.model_selection import train_test_split

In [81]:
# Cargar los datasets desde los archivos CSV
file_path_features = '/content/drive/MyDrive/SIS420 Inteligencia Artificial/SegundoSemestre/Dataset/features_v1'
file_path_target = '/content/drive/MyDrive/SIS420 Inteligencia Artificial/SegundoSemestre/Dataset/target'
data = pd.read_csv(file_path_features, header=0)  # 'header=0' asegura que la primera fila sea tratada como cabeceras


# Verificar que las cabeceras se hayan leído correctamente
print(data.head())  # Esto mostrará las primeras filas del dataset junto con los nombres de las columnas

# Eliminar la primera columna irrelevante
data = data.iloc[:, 1:]

# Cargar las clases (Y)
target_data = pd.read_csv(file_path_target)
y = target_data.iloc[:, -1]  # Solo la columna de clases (NObeyesdad)

   Unnamed: 0  Gender  family_history_with_overweight  FAVC  CAEC  SMOKE  SCC  \
0           0       0                               1     0     2      0    0   
1           1       0                               1     0     2      1    1   
2           2       1                               1     0     2      0    0   
3           3       1                               0     0     2      0    0   
4           4       1                               0     0     2      0    0   

   CALC  MTRANS       Age    Height    Weight  FCVC       NCP  CH2O       FAF  \
0     3       3  0.148936  0.320755  0.186567   0.5  0.666667   0.5  0.000000   
1     2       3  0.148936  0.132075  0.126866   1.0  0.666667   1.0  1.000000   
2     1       3  0.191489  0.660377  0.283582   0.5  0.666667   0.5  0.666667   
3     1       4  0.276596  0.660377  0.358209   1.0  0.666667   0.5  0.666667   
4     2       3  0.170213  0.622642  0.379104   0.5  0.000000   0.5  0.000000   

   TUE        BMI  
0  0.5

In [82]:
X = data
# Verificar si existen valores nulos
print("Valores nulos en el dataset:\n", data.isnull().sum())
# Verifica la estructura de X y y
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
m = y.size

Valores nulos en el dataset:
 Gender                            0
family_history_with_overweight    0
FAVC                              0
CAEC                              0
SMOKE                             0
SCC                               0
CALC                              0
MTRANS                            0
Age                               0
Height                            0
Weight                            0
FCVC                              0
NCP                               0
CH2O                              0
FAF                               0
TUE                               0
BMI                               0
dtype: int64
X shape: (22869, 17)
y shape: (22869,)


In [83]:
# Calcular el conteo de cada clase
clases, conteo = np.unique(y, return_counts=True)

# Imprimir el conteo de cada clase
for clase, count in zip(clases, conteo):
    print(f"Clase {int(clase)}: {count} ejemplos")

Clase 0: 2795 ejemplos
Clase 1: 3369 ejemplos
Clase 2: 2717 ejemplos
Clase 3: 2812 ejemplos
Clase 4: 3261 ejemplos
Clase 5: 3545 ejemplos
Clase 6: 4370 ejemplos


In [84]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [85]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [86]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [87]:
def CostFunction(theta, X, y, lambda_):
    m = y.size
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [88]:
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n))
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n)
        res = optimize.minimize(fun=CostFunction,
                                x0=initial_theta,
                                args=(X, (y == c).astype(int), lambda_),
                                method='CG',
                                jac=True,
                                options={'maxiter': 50})
        all_theta[c] = res.x
    return all_theta

In [90]:
def predictOneVsAll(all_theta, X):
    return np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)

# Agregar la columna de unos a X_normalizado
X_with_intercept = np.concatenate([np.ones((X_norm.shape[0], 1)), X_norm], axis=1)

# Entrenar el modelo
num_labels = 7
lambda_ = 0.1
all_theta = oneVsAll(X_with_intercept, y, num_labels, lambda_)

# Realizar predicciones
y_pred = predictOneVsAll(all_theta, X_with_intercept)

# Calcular la precisión
accuracy = np.mean(y_pred == y) * 100
print(f'Precisión del modelo One-vs-All: {accuracy:.2f}%')

Precisión del modelo One-vs-All: 76.01%


In [91]:
# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las características en el conjunto de entrenamiento
X_train_norm, mu, sigma = featureNormalize(X_train)

In [92]:
# Normalizar el conjunto de prueba usando los mismos parámetros (mu, sigma) del conjunto de entrenamiento
X_test_norm = (X_test - mu) / sigma

In [93]:
# Agregar la columna de unos (término de intercepción) para el conjunto de entrenamiento y prueba
X_train_with_intercept = np.concatenate([np.ones((X_train_norm.shape[0], 1)), X_train_norm], axis=1)
X_test_with_intercept = np.concatenate([np.ones((X_test_norm.shape[0], 1)), X_test_norm], axis=1)


In [94]:
# Entrenar el modelo con el conjunto de entrenamiento
all_theta = oneVsAll(X_train_with_intercept, y_train, num_labels, lambda_)

# Hacer predicciones en el conjunto de prueba
y_pred = predictOneVsAll(all_theta, X_test_with_intercept)

In [102]:
# Predicción para el conjunto de entrenamiento
y_pred_train = predictOneVsAll(all_theta, X_train_with_intercept)
train_accuracy = np.mean(y_pred_train == y_train) * 100
print(f'Precisión en el conjunto de entrenamiento: {train_accuracy:.2f}%')

# Predicción para el conjunto de prueba
y_pred_test = predictOneVsAll(all_theta, X_test_with_intercept)
test_accuracy = np.mean(y_pred_test == y_test) * 100
print(f'Precisión en el conjunto de prueba: {test_accuracy:.2f}%')


Precisión en el conjunto de entrenamiento: 76.11%
Precisión en el conjunto de prueba: 76.15%


In [97]:
# Calcular la precisión en el conjunto de prueba
accuracy = np.mean(y_pred == y_test) * 100
print(f'Precisión en el conjunto de prueba: {accuracy:.2f}%')

Precisión en el conjunto de prueba: 76.15%


In [98]:
# Comparar algunas predicciones con los valores reales (seleccionar los primeros 10 ejemplos)
n_muestras = 10
print("\nComparación de predicciones y valores reales (primeras 10 muestras):")
for i in range(n_muestras):
    print(f"Predicción: {y_pred[i]}, Valor real: {y_test.iloc[i]}")

# Mostrar las primeras 10 predicciones y valores reales
df_comparacion = pd.DataFrame({'Predicción': y_pred[:n_muestras], 'Valor real': y_test.iloc[:n_muestras]})
print(df_comparacion)


Comparación de predicciones y valores reales (primeras 10 muestras):
Predicción: 6, Valor real: 6
Predicción: 5, Valor real: 5
Predicción: 3, Valor real: 3
Predicción: 4, Valor real: 4
Predicción: 5, Valor real: 5
Predicción: 3, Valor real: 1
Predicción: 6, Valor real: 6
Predicción: 2, Valor real: 3
Predicción: 3, Valor real: 3
Predicción: 0, Valor real: 0
       Predicción  Valor real
9123            6           6
18200           5           5
11206           3           3
16463           4           4
9303            5           5
16326           3           1
3502            6           6
19228           2           3
1063            3           3
7660            0           0
